In [1]:
pip install openai gradio whisper pyttsx3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 6.9 MB/s eta 0:00:00
  Created wheel for whisper: filename=whisper-1.1.10-py3-none-any.whl size=41120 sha256=5ecf8e72cd2fa8f60ac423ee65806b08d571480255f12cfca65609f8af631fff
  Stored in directory: /root/.cache/pip/wheels/aa/7c/1d/015619716e2facae6631312503baf3c3220e6a9a3508cb14b6
Successfully built whisper
  Attempting uninstall: markupsafe
    

In [5]:
pip install gtts

In [14]:
import os
print(os.path.exists("calculus_training_data.jsonl"))

False


In [2]:
import whisper

def transcribe_audio(audio_path):
    model = whisper.load_model("base")
    result = model.transcribe(audio_path)
    return result["text"]

In [3]:
import openai

openai.api_key = "your_openai_api_key"

def get_response(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
    )
    return response["choices"][0]["message"]["content"]

In [6]:
from gtts import gTTS

def generate_audio(text, filename="response.mp3"):
    tts = gTTS(text)
    tts.save(filename)
    return filename

In [7]:
def voice_chat(audio_input):
    # Step 1: Convert voice to text
    text_input = transcribe_audio(audio_input)

    # Step 2: Query the LLM
    response = get_response(text_input)

    # Step 3: Convert text response to speech
    audio_output = generate_audio(response)

    return audio_output

In [10]:
import gradio as gr

def chat_with_assistant(audio_input=None, text_input=None):
    # If audio is provided, transcribe it
    if audio_input:
        user_query = transcribe_audio(audio_input)
    elif text_input:
        user_query = text_input
    else:
        return "Please provide a question either in text or audio.", None

    # Determine if it's a calculus question or conceptual query
    if "differentiate" in user_query or "integrate" in user_query:
        response = solve_calculus(user_query)
    else:
        response = query_llm(user_query)

    # Generate voice output
    audio_output = generate_audio(response)
    return response, audio_output

ui = gr.Interface(
    fn=chat_with_assistant,
    inputs=[
        gr.Audio(type="filepath", label="Ask with your voice"),
        gr.Textbox(lines=2, placeholder="Ask your question in text", label="Ask with text"),
    ],
    outputs=[
        gr.Textbox(label="Text Response"),
        gr.Audio(label="Voice Response"),
    ],
    title="TEACH ME: Calculus Teaching Assistant",
    description="Ask me calculus questions using voice or text, and I'll respond in both formats."
)

ui.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5dc664740c261620cb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
